In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import load_img, img_to_array
import matplotlib.pyplot as plt
import numpy as np
import os

# -----------------------------
# 0. GPU Setup
# -----------------------------
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU detected and memory growth enabled")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found, using CPU")

# -----------------------------
# Paths
# -----------------------------
train_dataset_path = r"D:\project\dataset\training_set"  # Folder with cats/ and dogs/
model_save_path = r"D:\project\dog_cat_model.h5"        # Path to save/load model
new_image_path = r"D:\project\cat.jpg"                 # Test image path

# -----------------------------
# 1. Check if model exists
# -----------------------------
if os.path.exists(model_save_path):
    # Load saved model
    model = load_model(model_save_path)
    print("Loaded existing model from disk.")
else:
    # -----------------------------
    # 2. Data Preprocessing
    # -----------------------------
    train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

    train_data = train_datagen.flow_from_directory(
        train_dataset_path,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary',
        subset='training'
    )

    val_data = train_datagen.flow_from_directory(
        train_dataset_path,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary',
        subset='validation'
    )

    # -----------------------------
    # 3. Build CNN Model
    # -----------------------------
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
        MaxPooling2D(2,2),

        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),

        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D(2,2),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Binary classification
    ])

    # -----------------------------
    # 4. Compile Model
    # -----------------------------
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # -----------------------------
    # 5. Train Model
    # -----------------------------
    history = model.fit(
        train_data,
        epochs=10,
        validation_data=val_data
    )

    # -----------------------------
    # 6. Plot Accuracy
    # -----------------------------
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.show()

    # -----------------------------
    # 7. Save Model
    # -----------------------------
    model.save(model_save_path)
    print(f"Model saved at {model_save_path}")

# -----------------------------
# 8. Test a New Image
# -----------------------------
img = load_img(r'D:\project\dc.jpg', target_size=(150,150))
img_array = img_to_array(img)/255.0
img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)

if prediction[0][0] > 0.5:
    print("It's a Dog 🐶")
else:
    print("It's a Cat 🐱")


No GPU found, using CPU
Loaded existing model from disk.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
It's a Dog 🐶
